In [ ]:
!pip install transformers --quiet

In [20]:
!id="1fdWdDm8LYw68kIZ6Z7xlT3Re8QqtLWis"; conf=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$id -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p'); wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$conf&id=$id" -O mmcovid_en.csv && rm -rf /tmp/cookies.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
--2022-08-25 13:40:49--  https://docs.google.com/uc?export=download&confirm=&id=1fdWdDm8LYw68kIZ6Z7xlT3Re8QqtLWis
Resolving docs.google.com (docs.google.com)... 142.250.186.206, 2a00:1450:401b:80d::200e
Connecting to docs.google.com (docs.google.com)|142.250.186.206|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-34-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5s8se6csd5q1pm9jf4qfko82i3unu6tk/1661427600000/05536367513713019519/*/1fdWdDm8LYw68kIZ6Z7xlT3Re8QqtLWis?e=download&uuid=259db83d-7360-4079-86c4-7e69043a1bd3 [following]
--2022-08-25 13:40:50--  https://doc-0o-34-docs.googleusercontent

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import RepeatedStratifiedKFold

data = pd.read_csv("mmcovid_en.csv",sep=",")
print("labels\n",data["label"].value_counts())
print("shape  \n",data.shape)

bootstrap_size = 0

if bootstrap_size != 0:
    bootstrap_factor = bootstrap_size / data.shape[0]
    bootstrap = np.random.uniform(size=data.shape[0]) < bootstrap_factor
    data = data.iloc[bootstrap]

    
X, y = data["text"].astype(str).values, data["label"].values

rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1410)
foldids = []
for fold_idx, (train, test) in tqdm(enumerate(rskf.split(X, y)), total=rskf.get_n_splits()):
    foldids.append((fold_idx,train,test))
    
print("shapes X",X.shape,"y", y.shape)

labels
 0    5304
1    2028
Name: label, dtype: int64
shape  
 (7332, 4)


100%|█████████████████████████████████████████| 10/10 [00:00<00:00, 1283.80it/s]

shapes X (7332,) y (7332,)


In [2]:
data.columns

Index(['Unnamed: 0', 'lang', 'label', 'text'], dtype='object')

In [2]:
import transformers
transformers.logging.set_verbosity_error()
from torch.utils.data import DataLoader 

from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import TFDistilBertModel, DistilBertTokenizerFast

from sklearn.base import BaseEstimator, TransformerMixin
import torch
import numpy as np

from scipy.sparse import csr_matrix
from tqdm import tqdm

from sklearn.linear_model import LogisticRegression

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("using device:", device)

if "disilbert_model" not in locals():
    disilbert_tokenizer =  AutoTokenizer.from_pretrained("distilbert-base-uncased")
    disilbert_model = AutoModel.from_pretrained("distilbert-base-uncased")
    handle = disilbert_model.to(device)


class BERTEmbeddings(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.tokenizer =  disilbert_tokenizer
        self.model = disilbert_tokenizer
        self.max_length = 256
        self.model_name = disilbert_model

    def fit(self, X=None, y=None):
        pass
    
    def encode(self, txt):
        return self.tokenizer(txt, max_length=self.max_length, 
                              truncation=True, padding=True, return_tensors="pt")

    def transform(self, X=None):
        dataloader = DataLoader(X, batch_size=4, shuffle=False)
        allembeds = []
        for batch in tqdm(dataloader):
            batchenc = disilbert_tokenizer(batch, max_length=256, truncation=True, padding=True, return_tensors="pt")
            input_ids = batchenc['input_ids'].to(device)
            attention_mask = batchenc['attention_mask'].to(device)
            batchout = disilbert_model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
            embeds = [vec[0].cpu().detach().numpy() for vec in batchout[1][-1]]
            allembeds.extend(embeds)
        return csr_matrix(allembeds)


class BertHead(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.head = LogisticRegression(class_weight='auto', max_iter=10000)

    def fit(self, X=None, y=None):
        self.head.fit(X, y)


    def transform(self, X=None):
        pass
    
    def predict(self, X=None):    
        return self.head.predict(X)



using device: cuda


In [3]:
bert = BERTEmbeddings()
X_dstil_numpy = bert.transform(X).toarray()

100%|██████████████████████████████████████| 1833/1833 [00:15<00:00, 115.74it/s]


In [5]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.linear_model import LogisticRegression

arr = []

for fold,j in enumerate(foldids):
    train = foldids[fold][1]
    test = foldids[fold][2]
    xin, yin = X_dstil_numpy[train], np.array(y[train])
    cls = BertHead()
    
    cls.fit(xin, yin)
    y_pred = cls.predict(X_dstil_numpy[test])

    bac = balanced_accuracy_score(y[test], y_pred)
    arr.append(bac)

    print(" BAC=", bac)

print(10*"-")
print("AVG. BAC=",np.mean(arr),"+/-",np.std(arr))

 BAC= 0.8763922728854856
 BAC= 0.8911706694512125
 BAC= 0.8927369764473837
 BAC= 0.8716498433693004
 BAC= 0.8888937231697412
 BAC= 0.8771899292261283
 BAC= 0.8851229841048845
 BAC= 0.8854710523262559
 BAC= 0.8838902424875276
 BAC= 0.8760877131917857
----------
AVG. BAC= 0.8828605406659704 +/- 0.006793676991393898
